## pitch2017前処理
### 右左で集計を分ける

In [76]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
pd.set_option('display.max_Columns', 100)

In [77]:
train_pitch = pd.read_feather('data/train_pitch.f')
print(train_pitch.shape)

(257117, 51)


In [78]:
OUT_PIT = 'intermediate/pit_2017_LR_1.f'
OUT_BAT = 'intermediate/bat_2017_5.f'

### 左右

In [79]:
train_pitch.replace('左', 'L', inplace=True)
train_pitch.replace('右', 'R', inplace=True)
train_pitch['pit_bat'] = train_pitch['投手投球左右'] + '_' + train_pitch['打者打席左右']

### 投手
#### 球種
0:ストレート 1:カーブ 2:スライダー 3:シュート 4:フォーク 5:チェンジアップ 6:シンカー 7:カットボール

In [80]:
train_pitch.replace({'球種': {
    0: '0_straight', 
    1: '1_curve', 
    2: '2_slider', 
    3: '3_shoot', 
    4: '4_fork', 
    5: '5_changeup', 
    6: '6_sinker', 
    7: '7_cutball'
}}, inplace=True)
train_pitch.rename(columns={'球種': 'ball'}, inplace=True)

In [81]:
pitch_ball = train_pitch[['投手ID','pit_bat','ball']]
pitch_ball = pd.get_dummies(pitch_ball, columns=['ball'])

In [82]:
groupby_pit = pitch_ball.groupby(['投手ID','pit_bat']).sum()
groupby_pit.rename(columns={
    'ball_0_straight': 'straight',
    'ball_1_curve': 'curve',
    'ball_2_slider': 'slider',
    'ball_3_shoot': 'shoot',
    'ball_4_fork': 'fork',
    'ball_5_changeup': 'changeup',
    'ball_6_sinker': 'sinker',
    'ball_7_cutball': 'cutball',
}, inplace=True)
pitch_ball = groupby_pit.reset_index(inplace=False)

In [83]:
pitch_ball['total'] = (pitch_ball['straight'] + pitch_ball['curve'] + pitch_ball['slider'] + pitch_ball['shoot']
                       + pitch_ball['fork'] + pitch_ball['changeup'] + pitch_ball['sinker'] + pitch_ball['cutball'])

In [84]:
pitch_ball.head()

,投手ID,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total
0,11606,R_L,134.0,54.0,0.0,0.0,61.0,0.0,0.0,13.0,262.0
1,11606,R_R,219.0,121.0,0.0,0.0,83.0,0.0,0.0,23.0,446.0
2,11766,L_L,111.0,0.0,103.0,67.0,0.0,0.0,2.0,0.0,283.0
3,11766,L_R,103.0,0.0,104.0,42.0,0.0,11.0,30.0,0.0,290.0
4,11807,R_L,249.0,38.0,1.0,9.0,115.0,0.0,0.0,2.0,414.0


### 登板試合数

In [85]:
pit_game = train_pitch[['投手ID', '試合ID']].groupby(['投手ID', '試合ID']).count()
pit_game = pd.DataFrame(pit_game.groupby(['投手ID']).size())
pit_game.reset_index(inplace=True)
pit_game.rename(columns={0: 'pit_game_cnt'}, inplace=True)

### イニング数

In [86]:
pit_inning = train_pitch[['投手ID', '試合ID', 'イニング']].groupby(['投手ID', '試合ID', 'イニング']).count()
pit_inning = pd.DataFrame(pit_inning.groupby(['投手ID']).size())
pit_inning.reset_index(inplace=True)
pit_inning.rename(columns={0: 'pit_inning_cnt'}, inplace=True)

### 対戦打者数

In [87]:
pit_batcnt = train_pitch[['投手ID', 'pit_bat', '試合ID', 'イニング', 'イニング内打席数']].groupby(['投手ID', 'pit_bat', '試合ID', 'イニング', 'イニング内打席数']).count()
pit_batcnt = pd.DataFrame(pit_batcnt.groupby(['投手ID','pit_bat']).size())
pit_batcnt.reset_index(inplace=True)
pit_batcnt.rename(columns={0: 'pit_batter_cnt'}, inplace=True)

### 投手実績まとめ

In [88]:
pitch_ball = pitch_ball.merge(pit_game, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_inning, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_batcnt, on=['投手ID','pit_bat'], how='left')

In [89]:
print(pitch_ball.shape)
pitch_ball.head()

(651, 14)


,投手ID,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
0,11606,R_L,134.0,54.0,0.0,0.0,61.0,0.0,0.0,13.0,262.0,46,54,63
1,11606,R_R,219.0,121.0,0.0,0.0,83.0,0.0,0.0,23.0,446.0,46,54,108
2,11766,L_L,111.0,0.0,103.0,67.0,0.0,0.0,2.0,0.0,283.0,50,50,73
3,11766,L_R,103.0,0.0,104.0,42.0,0.0,11.0,30.0,0.0,290.0,50,50,81
4,11807,R_L,249.0,38.0,1.0,9.0,115.0,0.0,0.0,2.0,414.0,52,61,95


In [90]:
pitch_ball.describe()

,投手ID,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
count,6.510000e+02,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000
mean,1.162924e+06,184.940092,29.513057,73.385561,27.897081,32.786482,21.413210,4.486943,20.534562,394.956989,20.797235,50.321045,100.001536
std,4.160398e+05,190.605887,58.372104,94.297583,68.452185,58.598626,55.396277,30.041577,57.853821,391.279901,18.815592,47.493933,99.171499
min,1.160600e+04,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,9.004290e+05,40.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.000000,5.500000,11.500000,22.000000
50%,1.300036e+06,126.000000,5.000000,33.000000,0.000000,4.000000,0.000000,0.000000,0.000000,276.000000,15.000000,37.000000,71.000000
75%,1.500109e+06,254.000000,32.000000,113.000000,17.000000,37.500000,11.000000,0.000000,4.000000,520.000000,27.000000,68.000000,130.500000
max,1.700016e+06,1083.000000,458.000000,710.000000,637.000000,331.000000,438.000000,451.000000,475.000000,1886.000000,72.000000,190.000000,471.000000


In [91]:
pitch_ball.to_feather(OUT_PIT)

## 野手
### 打席数

In [92]:
bat_ball = train_pitch[['打者ID', '試合ID', 'イニング', 'イニング内打席数']].groupby(['打者ID', '試合ID', 'イニング', 'イニング内打席数']).count()
bat_ball = pd.DataFrame(bat_ball.groupby(['打者ID']).size())
bat_ball.reset_index(inplace=True)
bat_ball.rename(columns={0: 'batter_cnt'}, inplace=True)

### 試合数

In [93]:
bat_game = train_pitch[['打者ID', '試合ID']].groupby(['打者ID', '試合ID']).count()
bat_game = pd.DataFrame(bat_game.groupby(['打者ID']).size())
bat_game.reset_index(inplace=True)
bat_game.rename(columns={0: 'bat_game_cnt'}, inplace=True)

### 打者成績まとめ

In [94]:
bat_ball = bat_ball.merge(bat_game, on='打者ID', how='left')

In [95]:
print(bat_ball.shape)
bat_ball.head()

(466, 3)


,打者ID,batter_cnt,bat_game_cnt
0,11270,83,35
1,11302,156,69
2,11343,41,28
3,11436,268,81
4,11511,48,21


In [96]:
bat_ball.describe()

,打者ID,batter_cnt,bat_game_cnt
count,4.660000e+02,466.000000,466.000000
mean,1.044992e+06,139.688841,41.448498
std,4.687117e+05,184.754006,45.398623
min,1.127000e+04,1.000000,1.000000
25%,8.000125e+05,7.250000,4.000000
50%,1.100114e+06,44.000000,21.000000
75%,1.400182e+06,222.500000,71.750000
max,1.700019e+06,680.000000,143.000000


In [97]:
bat_ball.to_feather(OUT_BAT)